# colab을 이용한 기계학습 실습

🎯 학습 목표 : colab 환경에서 기계 학습 코드를 실행하고 결과를 확인할 수 있다.

- 실습 재료

| 항목 | 상세 |
| ---- | ---- |
| 🗂️ 데이터 | AI-HUB의 치매 고위험군 웨어러블 라이프로그 중 샘플 데이터 |
| 🤖 기계 학습 알고리즘 | 결정 트리 (Decision Tree) |
| 🏗️ 기계 학습 프레임워크 | Scikit-learn |
| 🐍 프로그래밍 언어 | Python |
| 👩‍💻 프로그래밍 환경 | Colab |


- colab에서 코드 실행 방법은 다음 그림을 참조해주시기 바랍니다.
    ![](https://i.imgur.com/0GoFr7q.png)

<!-- ## 1. Colab이란?

- colab은 구글에서 제공하는 온라인 기계 학습 환경 입니다.

![](https://post-phinf.pstatic.net/MjAxOTEwMTVfMTE4/MDAxNTcxMTA3ODE4NTcy.KmzXuRSS3HWe4qnBR7esUkTOCPELkbi6fD0khAX8i8kg.9KgltPv7JsznlhiQVmmCxwVFBqLUI03VZaAwFwxUfHkg.JPEG/29.JPG?type=w1200)
*(그림 출처 : [네이버 블로그](https://post.naver.com/viewer/postView.nhn?volumeNo=26447765))*

- 기계 학습 코드를 작성하고, 실행할 수 있습니다.
  - 기계 학습 코드를 작성하기 위해서 필요한 것은 웹 브라우저뿐 입니다! 지금 여러분들이 보고 계시듯이 말이죠.
  - 실행은 구글의 데이터 센터에서 실행됩니다. -->

### 1. 필요 라이브러리 및 함수 추가

원할한 실습을 위해 필요한 데이터 변환 및 분석에 사용되는 함수를 미리 만들어 두었습니다. 😉

- *(참고) 데이터 변환 및 사용되는 함수 코드는 https://github.com/taeminlee/ai_practice 에서 모두 확인 가능합니다.*

실습에 앞서 아래의 코드 셀을 실행해주시기 바랍니다.

In [ ]:
# 이번 실습에 사용하는 데이터 및 함수를 다운로드 받습니다.
## 참고 : 느낌표(!)로 시작하는 코드는 python이 아닌, linux 운영체제 명령어로 실행됩니다.
!rm ai_practice -rf
!git clone https://github.com/taeminlee/ai_practice
# 데이터를 불러옵니다.
## 참고 : 해당 데이터 처리 코드는 https://github.com/taeminlee/ai_practice/data_util.py 에 기록되어 있습니다.
from ai_practice.data_util import data, Q_dict, X, Y, train_X, train_Y, test_X, test_Y
# 분석 함수를 불러옵니다.
## 참고 : 해당 함수 코드는 https://github.com/taeminlee/ai_practice/model_util.py 에 기록되어 있습니다.
from ai_practice.model_util import get_graph, plot_feature_importance, get_feature_importance_table
# 기계 학습 라이브러리를 불러옵니다.
from sklearn.tree import DecisionTreeClassifier

### 2. 데이터 살펴보기

이번 실습에서 사용되는 데이터는 [AI-HUB의 치매 고위험군 웨어러블 라이프로그 데이터 집합](https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=226) 중 인지기능 데이터의 샘플 자료를 이용합니다.

실습 데이터 집합에 대한 명세는 다음과 같습니다.

- 데이터 개수 : 16개
- 주요 항목
  - DIAG_NM : 진단명
    - (CN : `정상`, MCI : `경도인지장애`, Dem : `치매`)
  - Q01 ~ Q19 : 부분 질문을 포함하여, 총 30개의 질문
    - (정답 : `2`, 오답 : `1`)
  - TOTAL : 총점
    - (0 ~ 30)

In [ ]:
# 데이터를 살펴보기 위해 이 코드 블록을 실행시켜 주십시오.
data

30개의 문항에 대한 상세 정보는 다음과 같습니다.

In [ ]:
# 문항 확인을 위해 이 코드 블록을 실행시켜 주십시오.
Q_dict

### 3. 데이터 전처리

학습을 하기 위해서는 입력 값으로 쓸 부분과, 결과 값으로 쓸 부분을 나누어야 합니다.

이번 실습에서는 
- `입력 값`으로 Q01 ~ Q19의 30개의 `문항 결과`를, 
- `결과 값`으로 Diag_nm의 `진단명`으로 나누어 진행합니다.

In [ ]:
# 입력 값 확인을 위해 코드 블록을 실행해 주십시오.
X

In [ ]:
# 결과 값 확인을 위해 코드 블록을 실행해 주십시오.
Y

그리고, 학습에 쓸 데이터 집합과 테스트에 쓸 데이터 집합으로 나누어야 합니다.

본 실습에서는 10번과 12번 데이터를 테스트 집합으로 사용합니다.
-  10번이 12번에 비해 총점은 높지만, `정상`이 아닌 `경도인지장애`를 겪고 있기 때문에, 어느정도 합당한 테스트가 될 것입니다.

![](https://i.imgur.com/gAMPFkY.png)

In [ ]:
# 학습 데이터 집합의 입력 값 확인을 위해 코드 블록을 실행해 주십시오.
train_X

In [ ]:
# 학습 데이터 집합의 결과 값 확인을 위해 코드 블록을 실행해 주십시오.
train_Y

In [ ]:
# 테스트 데이터 집합의 입력 값 확인을 위해 코드 블록을 실행해 주십시오.
test_X

In [ ]:
# 테스트 데이터 집합의 결과 값 확인을 위해 코드 블록을 실행해 주십시오.
test_Y

### 4. 모델 학습

본 실습에서는 결정 트리 분류기 알고리즘으로 모델을 학습할 것 입니다.

결정 트리 알고리즘은 [scikit-learn](https://scikit-learn.org/) 라이브러리를 이용합니다.

본 실습에서는 결정 트리를 학습할 것이므로, scikit-learn 중 tree 알고리즘 중 DecisionTreeClassifier 를 사용합니다.

*(참고) 그 외 기계학습 알고리즘은 [scikit-learn 홈페이지의 user guide 페이지](https://scikit-learn.org/stable/supervised_learning.html#supervised-learning) 에서 확인 가능합니다.*

scikit-learn 라이브러리에서 학습은 `.fit(X, Y)` 함수를 이용하여 쉽게 수행하도록 구성되어 있습니다.

다음의 코드는 결정 트리 분류기를 생성하고, 학습하는 과정입니다.

In [ ]:
# scikit-learn 라이브러리에서 결정 트리 분류기를 불러옵니다.
from sklearn.tree import DecisionTreeClassifier

# 결정 트리 분류기 객체를 생성합니다.
clf = DecisionTreeClassifier()
# train_X 집합을 입력값, train_Y 집합을 결과값으로 모델을 학습합니다.
clf = clf.fit(train_X,train_Y)
# 결과 모델은 clf 변수에 저장되어 있습니다.

scikit-learn은 학습 성능 평가와 관련된 기능 역시 제공합니다.

본 실습에서는 분류 모형의 일반적인 결과를 모아서 보여주는 `classification_report(pred, Y)` 함수를 이용합니다.

*(참고) 그 외 평가 함수는 [scikit-learn 홈페이지의 user guide 페이지](https://scikit-learn.org/stable/modules/model_evaluation.html)에서 확인 가능합니다.*

해당 함수는 `모델의 예측값(pred)`과 `정답값(Y)`의 2개의 입력 변수가 필요합니다.

- 모델의 예측 값은 학습이 끝난 모델에서 `.predict(X)` 함수를 수행하여 얻을 수 있습니다.

- 정답값은 데이터 집합 중 결과 값을 그대로 이용합니다.

다음의 코드는 학습된 모델의 성능을 평가합니다.

In [ ]:
# scikit-learn 라이브러리에서 분류 모형 평가 도구를 불러옵니다.
from sklearn.metrics import classification_report

# 학습 데이터 집합에 대한 예측 값을 생성합니다.
train_pred = clf.predict(train_X)

# 학습 데이터 집합에 대한 성능 평가를 수행합니다.
print("*" * 15 + " TRAIN DATASET REPORT " + "*" * 16)
print(classification_report(train_pred, train_Y))

# 테스트 데이터 집합에 대한 예측 값을 생성합니다.
test_pred = clf.predict(test_X)

# 테스트 데이터 집합에 대한 성능 평가를 수행합니다.
print("*" * 16 + " TEST DATASET REPORT " + "*" * 16)
print(classification_report(test_pred, test_Y))

이상으로 본 실습을 마치도록 하겠습니다.

모두들 고생 많으셨습니다!

![](https://img.favpng.com/10/1/7/kakaotalk-kakao-friends-emoticon-sticker-png-favpng-mZm2vp0mk2Ce9aTUnBjC4s4DZ.jpg)

### One more thing. 모델 분석하기

일부 기계학습 모형의 경우 학습이 완료한 모형이 어떻게 구조화 되어 있는지 확인이 가능합니다.

본 실습에서는 결정 트리를 시각화 하여 살펴보도록 하겠습니다.

In [ ]:
# 분석 함수를 불러옵니다.
## 참고 : 해당 함수 코드는 https://github.com/taeminlee/ai_practice/model_util.py 에 기록되어 있습니다.
from ai_practice.model_util import get_graph, plot_feature_importance, get_feature_importance_table

# 결정 트리를 시각화 합니다.
get_graph(clf)

In [ ]:
# feature 중요도 그래프를 출력합니다.
plot_feature_importance(clf)

In [ ]:
# feature 중요도를 표 형식으로 살펴봅니다.
get_feature_importance_table(clf)